In [ ]:
# Packages to be imported

import random
import math
import csv
import os
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Dense
from ipynb.fs.full.Functions import *

In [ ]:
# Parameter Initialization
time_step =[]
Mbits_transmitted = []

# Data for training.
with open('./Traffic_Train_Data.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter = ',')
    next(reader)
    for row in reader:
        Mbits_transmitted.append(float(row[1]))
        time_step.append(int(row[0]))

#converting the lists into arrays
series_trans = np.array(Mbits_transmitted)
time = np.array(time_step)
Clients = create_clients(series_trans,35, 'clients')

In [ ]:
# parameters
poly_mod_degree = 8192
coeff_mod_bit_sizes = [40, 20, 40]
# create TenSEALContext
ctx_eval = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
# scale of ciphertext to use
ctx_eval.global_scale = 2 ** 30
# this key is needed for doing dot-product operations
ctx_eval.generate_galois_keys()

In [ ]:
#initialize global model
smlp_global = SimpleMLP()
global_model = smlp_global.build()
        
#commence global training loop
for comm_round in range(comms_round):
            
    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()
    
    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()
    Noise = list()
    #randomize client data - using keys
    client_names= list(Clients)
    random.shuffle(client_names)
    
    #loop through each client and create new local model
    for client in client_names:
        window_size = 20
        batch_size = 16
        dataset = windowed_dataset(np.array(Clients[client]), window_size, batch_size, len(Clients[client]))        
        smlp_local = SimpleMLP()
        local_model = smlp_local.build()
        #compiling the model with a mean squared error loss and a stochastic gradient descent optimizer
        local_model.compile(loss="mse", optimizer=tf.keras.optimizers.SGD(learning_rate=3e-4, momentum=0.9), metrics=["mae"])
                
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)
        
        #fit local model with client's data
        local_model.fit(dataset, epochs=1, verbose=0)
        
        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(Clients, client)
        model_encrypted = encrypt_weights(local_model,scaling_factor, ctx_eval)
        noisy_model_weight, noise = add_noise(model_encrypted, ctx_eval)
        Noise.append(noise)
        scaled_local_weight_list.append(noisy_model_weight)
        
        #clear session to free memory after each communication round
        K.clear_session()
        
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = decrypt_weights((remove_noise(sum_enc_weights(scaled_local_weight_list),Noise, ctx_eval)),ctx_eval)
    
    #update global model 
    global_model.set_weights(average_weights)



In [ ]:
# For testing the performance of the model.
with open('./Traffic_Test_Data.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter = ',')
    next(reader)
    for row in reader:
        Mbits_transmitted.append(float(row[1]))
        time_step.append(int(row[0]))

#converting the lists into arrays
series_trans_test = np.array(Mbits_transmitted)

# The average MSE on 9 datasets
print(get_average_mse(series_trans_test, global_model, window_size, 10))